In [6]:
import sqlite3
import json
import numpy as np
from ase import io, Atoms
from ase.build import molecule
from collections import Counter, defaultdict
from scipy.spatial import distance_matrix
from itertools import combinations
from openbabel.pybel import readfile
from sympy import Plane, Point3D

con = sqlite3.connect('ligands.db')
cur = con.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS ligand_data
              (ccdc_id text, denticity integer, coordinating_elements text, coordinating_indices text,
               original_metal integer, xyz text)'''
          )

In [7]:
import math
from math import pi ,sin, cos

def dotproduct(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def length(v):
    return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
    return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))

def R(theta, u):
    return [[cos(theta) + (u[0]**2) * (1-cos(theta)), 
             u[0] * u[1] * (1-cos(theta)) - u[2] * sin(theta), 
             u[0] * u[2] * (1 - cos(theta)) + u[1] * sin(theta)],
            [u[0] * u[1] * (1-cos(theta)) + u[2] * sin(theta),
             cos(theta) + (u[1]**2) * (1-cos(theta)),
             u[1] * u[2] * (1 - cos(theta)) - u[0] * sin(theta)],
            [u[0] * u[2] * (1-cos(theta)) - u[1] * sin(theta),
             u[1] * u[2] * (1-cos(theta)) + u[0] * sin(theta),
             cos(theta) + (u[2]**2) * (1-cos(theta))]]

def check_planarity(ligand_positions, ligand_connect_indices):
    plane = Plane(Point3D(list(ligand_positions[ligand_connect_indices[0]])), 
                  Point3D(list(ligand_positions[ligand_connect_indices[1]])), 
                  Point3D(list(ligand_positions[ligand_connect_indices[2]])))
    point2plane = []
    for idx, point in enumerate(ligand_positions):
        if idx in ligand_connect_indices:
            continue
        dist = plane.distance(Point3D(point))
        point2plane.append(float(dist))
    return point2plane
    

def Rotate(pointsToRotate, point2, theta):
    """
    adapted from: https://stackoverflow.com/questions/17763655/rotation-of-a-point-in-3d-about-an-arbitrary-axis-using-python
    needs to be origin
    rotates given:
    pointToRotate: the atoms to rotate around the axis
    point2: the coordinates to define one of the axes (eg. the metal)
    theta: the angle to rotate through
    """
    diff = -point2
    # make it unit norm
    u = diff / np.linalg.norm(diff)
    r = R(theta, u)
    rotated = []
    orig_shape = pointsToRotate.shape
    for point in pointsToRotate:
        for i in range(3):
            rotated.append(sum([r[j][i] * point[j] for j in range(3)]))
    
    return list(np.array(rotated).reshape(orig_shape))

In [8]:
pyr_atoms = [7, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1]
pyr_pos = np.array([[0.6816, 1.1960, 0.0000],
           [1.3603, 0.0256, 0.0000],
           [0.6971, -1.2020, 0.0000],
           [-0.6944, -1.2184, 0.0000],
           [ -1.3895, -0.0129, 0.0000],
           [-0.6712, 1.1834, 0.0000],
           [2.4530, 0.1083, 0.0000],
           [ 1.2665, -2.1365, 0.0000],
           [-1.2365, -2.1696, 0.0000],
           [-2.4837, 0.0011, 0.0000],
           [-1.1569, 2.1657, 0.0000]
          ])
pyr_pos[:,0] = pyr_pos[:,0]-0.6816
pyr_pos[:,1] = pyr_pos[:,1]-1.1960
pyridine = Atoms(symbols=pyr_atoms,
                 positions=pyr_pos)
print(len(pyridine))

11


In [11]:
string = """select ccdc_id, denticity, coordinating_elements, coordinating_indices, xyz from ligand_data 
            where (coordinating_elements like '%[8]%' or coordinating_elements like '%[7]%' 
            or coordinating_elements like '%[8, 7]%' or coordinating_elements like '%[7, 8]%') 
            and (denticity=3) and (original_metal=25 or original_metal=26 or original_metal=27 or original_metal=28)"""
# string = """select ccdc_id, denticity, coordinating_elements, coordinating_indices, xyz from ligand_data 
#             where (coordinating_elements like '%[7]%') 
#             and (denticity=3) and (original_metal=25 or original_metal=26 or original_metal=27 or original_metal=28)"""

count = 0 

def min_distance(coords, ligand_len):
    return np.mean(distance_matrix(coords[-ligand_len+1:], coords[:len(coords)-ligand_len]).flatten())

def add2tridentate(tri_ligand_xyz_str, connecting_indices, tmqm_id, ligand2add):
    """
    Args:
    tri_ligand_xyz_str: A string of the xyz of the separated tridentate ligand.
    connecting_indices: The indices of atoms in the ligand that connect to the metal.
    tmqm_id: The id stored in the ccdc
    ligand2add: Ase Atoms object, with the 0th index referring to the connecting point.
    """
    with open("tmp__.xyz", "w") as f:
        f.write(tri_ligand_xyz_str)
    ligand = io.read("tmp__.xyz")
    connecting_positions = np.zeros(shape=(3, 3))
    for idx, conn_idx in enumerate(connecting_indices):
        connecting_positions[idx] = ligand.positions[conn_idx]
    # define the vectors that make up the tridentate triangle
    vec1 = connecting_positions[0]-connecting_positions[1]
    vec2 = connecting_positions[1]-connecting_positions[2]
    vec3 = connecting_positions[0]-connecting_positions[2]
    # need mapping to determine which vertex of the triangle
    # is the point at which we have the highest angle.
    # The key is the index of the angles list, the value is the vertex index,
    # eg. vec1 and vec2 shre the index 1, so that is the intersection point.
    angle_idx2_vertex_idx = {0: 1, 1: 0, 2: 2}
    angles = [angle(-vec1, vec2), angle(vec1, vec3), angle(vec2, vec3)]
    sum_angles = sum(angles)
    opposite_atom = connecting_positions[angle_idx2_vertex_idx[np.argmax(angles)]]
    assert abs(sum_angles-3.141)<0.001
    angle_ratio = max(angles)/min(angles)
    if angle_ratio<1.4:
        # assume it is meriodonic ligand
        # TO DO: Handle meriodonic ligands.
        # I suppose since none of the atoms are opposite eachother,
        #  we could just take any atom.
        return

    tmqm_xyz = io.read("all_relevant_xyzs/{}.xyz".format(tmqm_id))
    metal_coord = tmqm_xyz.positions[np.argmax(tmqm_xyz.numbers)]
    
    # sanity check the position of atoms
    connecting_distances = set(distance_matrix(connecting_positions, connecting_positions).flatten())
    # remove the 0 value since atoms are obviously 0 Angstroms from themselves
    connecting_distances.remove(0)
    min_dist = min(connecting_distances)
    sum_distances = 0
    for dist in metal_coord-connecting_positions:
        sum_distances+=(np.linalg.norm(dist))
    avg_dist = sum_distances/3
    if avg_dist>min_dist:
        #  The connecting atoms are in some cases closer to eachother than the metal
        return 

    # add the tmqm ligand
    positions = np.append([metal_coord], ligand.positions, 0)
    # translate the original ligand xyz to the new position
    # define the direction at which we will add the atom
    metal2opposite = metal_coord-opposite_atom
    ligand_conn_pos = opposite_atom+2*metal2opposite
    ligand_positions = ligand2add.positions
    ligand_atoms = ligand2add.numbers
    translate_lig = ligand_positions[0]-ligand_conn_pos
    lig_pos = ligand_positions-translate_lig
    # add the new ligand
    pos_w_ligand = np.append(positions, lig_pos, 0)
    # make connecting point the origin
    pos_w_ligand = pos_w_ligand-lig_pos[0]
    angle_range = np.arange(0, 3.14, 0.2)
    # define other axes
    other_ax = np.array([pos_w_ligand[0][1], -pos_w_ligand[0][0], pos_w_ligand[0][2]])
    final_ax = np.array([pos_w_ligand[0][0], pos_w_ligand[0][2], -pos_w_ligand[0][1]])
    # first axis is defined as the position of the metal, since we set the 
    # connecting atom as the origin
    axes_def = (pos_w_ligand[0], other_ax, final_ax)
    min_dists = [0]
    print(lig_pos[0])
    for angles in combinations(angle_range, 3):
        pos2rotate = pos_w_ligand
        pos2rotate[-len(ligand2add)+1:] = Rotate(pos2rotate[-len(ligand2add)+1:], axes_def[0], angles[0])
        pos2rotate[-len(ligand2add)+1:] = Rotate(pos2rotate[-len(ligand2add)+1:], axes_def[1], angles[1])
        pos2rotate[-len(ligand2add)+1:] = Rotate(pos2rotate[-len(ligand2add)+1:], axes_def[2], angles[2])
        min_dist = min_distance(pos2rotate, len(ligand2add))
        if min_dist>max(min_dists):
            thing = Atoms([26]+list(ligand.numbers)+list(ligand2add.numbers), positions=pos2rotate)
            thing.write("save_as_opt.xyz")
            
for row in cur.execute(string):
    xyz_str = row[4]
    add2tridentate(row[4], json.loads(row[3]), row[0], pyridine)
    print('iterating over the function def for: {}'.format(row[0]))

for row in cur.execute(string):    
    xyz_str = row[4]
    print(row[0], row[1])
    with open("tmp__.xyz", "w") as f:
        f.write(xyz_str)
    ligand = io.read("tmp__.xyz")
    connecting_indices = json.loads(row[3])
    connecting_positions = np.zeros(shape=(row[1], 3))
    for idx, conn_idx in enumerate(connecting_indices):
        connecting_positions[idx] = ligand.positions[conn_idx]
    # define the vectors that make up the tridentate triangle
    vec1 = connecting_positions[0]-connecting_positions[1]
    vec2 = connecting_positions[1]-connecting_positions[2]
    vec3 = connecting_positions[0]-connecting_positions[2]
    # need mapping to determine which vertex of the triangle
    # is the point at which we have the highest angle.
    # The key is the index of the angles list, the value is the vertex index,
    # eg. vec1 and vec2 shre the index 1, so that is the intersection point.
    angle_idx2_vertex_idx = {0: 1, 1: 0, 2: 2}
    angles = [angle(-vec1, vec2), angle(vec1, vec3), angle(vec2, vec3)]
    sum_angles = sum(angles)
    opposite_atom = connecting_positions[angle_idx2_vertex_idx[np.argmax(angles)]]
    assert abs(sum_angles-3.141)<0.001
    angle_ratio = max(angles)/min(angles)
    if angle_ratio<1.4:
        # assume it is meriodonic ligand
        # TO DO: Handle meriodonic ligands.
        # I suppose since none of the atoms are opposite eachother,
        #  we could just take any atom.
        continue

    tmqm_xyz = io.read("all_relevant_xyzs/{}.xyz".format(row[0]))
    metal_coord = tmqm_xyz.positions[np.argmax(tmqm_xyz.numbers)]
    
    # sanity check the position of atoms
    connecting_distances = set(distance_matrix(connecting_positions, connecting_positions).flatten())
    # remove the 0 value since atoms are obviously 0 Angstroms from themselves
    connecting_distances.remove(0)
    min_dist = min(connecting_distances)
    sum_distances = 0
    for dist in metal_coord-connecting_positions:
        sum_distances+=(np.linalg.norm(dist))
    avg_dist = sum_distances/row[1]
    if avg_dist>min_dist:
        #  The connecting atoms are in some cases closer to eachother than the metal
        continue

    # add the tmqm ligand
    positions = np.append([metal_coord], ligand.positions, 0)
    # translate the original ligand xyz to the new position
    # define the direction at which we will add the atom
    metal2opposite = metal_coord-opposite_atom
    ligand_conn_pos = opposite_atom+2*metal2opposite
    translate_pyr = pyr_pos[0]-ligand_conn_pos
    pyridine_pos = pyr_pos-translate_pyr
    # add the new ligand
    pos_w_ligand = np.append(positions, pyridine_pos, 0)
    pos_w_ligand = pos_w_ligand-pyridine_pos[0]
    angle_range = np.arange(0, 3.14, 0.2)
    # define other axes
    other_ax = np.array([pos_w_ligand[0][1], -pos_w_ligand[0][0], pos_w_ligand[0][2]])
    final_ax = np.array([pos_w_ligand[0][0], pos_w_ligand[0][2], -pos_w_ligand[0][1]])
    # first axis is defined as the position of the metal, since we set the 
    # connecting atom as the origin
    axes_def = (pos_w_ligand[0], other_ax, final_ax)
    min_dists = [0]
    for angles in combinations(angle_range, 3):
        pos2rotate = pos_w_ligand
        pos2rotate[-len(pyridine)+1:] = Rotate(pos2rotate[-len(pyridine)+1:], axes_def[0], angles[0])
        pos2rotate[-len(pyridine)+1:] = Rotate(pos2rotate[-len(pyridine)+1:], axes_def[1], angles[1])
        pos2rotate[-len(pyridine)+1:] = Rotate(pos2rotate[-len(pyridine)+1:], axes_def[2], angles[2])
        min_dist = min_distance(pos2rotate, len(pyridine))
        if min_dist>max(min_dists):
            thing = Atoms([26]+list(ligand.numbers)+pyr_atoms, positions=pos2rotate)
            thing.write("save_as_opt.xyz")
        
    count+=1
    #print(planarness)
    #break
print(count)
plt.hist(mean_planarities)

[ 6.03940719 10.03304021  8.48141687]
iterating over the function def for: ZAJGUP
[14.62298244  8.60958063  6.38018096]
iterating over the function def for: YAVKOY
iterating over the function def for: DOPXIQ
[-3.43370259e-05  1.02559911e+01 -1.80167408e+00]
iterating over the function def for: XECVEJ
iterating over the function def for: RANFIV
[ 0.51817299  3.87169137 15.21122948]
iterating over the function def for: WIJYAQ
[4.48802286 4.57864686 9.02497705]
iterating over the function def for: DOCWUO
[14.43422657 12.33564302  7.50746194]
iterating over the function def for: MEGZOP
iterating over the function def for: GAJJAD
iterating over the function def for: FEHFOO
[-1.5763098   2.02121711  7.30969937]
iterating over the function def for: HEBBEW
iterating over the function def for: IGUWIR
iterating over the function def for: LOCZUY
[ 6.14798412  5.86655112 14.17130934]
iterating over the function def for: LETKAX
[ 5.72526103  0.42310937 -0.85237105]
iterating over the function def f

iterating over the function def for: JACKOQ
[-4.63755565  4.95276453 15.37084247]
iterating over the function def for: AYEDAK
[1.14089199 1.73029002 2.7958265 ]
iterating over the function def for: VUCYUO
iterating over the function def for: ANAJUW
[ 7.75683013 10.86409426 11.19534896]
iterating over the function def for: PAZBUQ
iterating over the function def for: FEHFUU
iterating over the function def for: UXOBEQ
[0.93571785 6.98225759 1.73447757]
iterating over the function def for: ICIRIX
[3.04739097 4.7538956  0.87056565]
iterating over the function def for: HEZMUW
iterating over the function def for: ZOVPAB
iterating over the function def for: ETIXIP
iterating over the function def for: GISWIR
[ 2.34097458 10.05857793  5.42284559]
iterating over the function def for: QADPOA
iterating over the function def for: CIGVAR
[ 1.2584036  23.05737483  3.55102404]
iterating over the function def for: RATXOB
[ 0.74998108 -0.34244582  1.67027252]
iterating over the function def for: QIVVIA
[

iterating over the function def for: BULYEM
[7.36531275 6.15855519 4.83841873]
iterating over the function def for: PAYTOZ
iterating over the function def for: AQIREZ
[ 2.94073749  0.40125301 15.08575495]
iterating over the function def for: DUWGUZ
[ 8.08338194 11.11556396  1.98091227]
iterating over the function def for: SORJEP
[-0.25218244 -1.09810676  3.18711126]
iterating over the function def for: BEXFER
[ 0.94453738  6.70984735 12.38672394]
iterating over the function def for: LANTUS
[ 3.05777755 -0.90384673  1.95182153]
iterating over the function def for: QOHBOF
[3.1762474  2.12635611 4.06298168]
iterating over the function def for: GIVSEK
[6.8770365  4.21538732 0.88999221]
iterating over the function def for: SUGXOJ
[5.78246425 1.24811558 1.18399458]
iterating over the function def for: NOWVIE
iterating over the function def for: QIFRAY
[1.76858489 3.44600636 7.31414015]
iterating over the function def for: QERZOC
[ 0.91900339 11.26343897  9.83858082]
iterating over the functi

KeyboardInterrupt: 

In [ ]:
from openbabel import openbabel


def run_ff(filename)
    obConversion = openbabel.OBConversion()
    obConversion.SetInFormat("xyz")

    OBMol = openbabel.OBMol()
    obConversion.ReadFile(OBMol, "save_as_opt_axial.xyz")

    OBMol_thing = OBMol

    metals = list(range(21, 31))+list(range(39, 49))+list(range(72, 81))

    constr = openbabel.OBFFConstraints()
    # openbabel indexing starts at 1 ### !!!
    # convert metals to carbons for FF
    indmtls = []
    mtlsnums = []
    for iiat, atom in enumerate(openbabel.OBMolAtomIter(OBMol_thing)):
        if atom.GetAtomicNum() in metals:
            indmtls.append(iiat)
            mtlsnums.append(atom.GetAtomicNum())
            atom.SetAtomicNum(6)
    for cat in range(0, len(pos_w_ligand)-len(pyridine)):
        constr.AddAtomConstraint(cat+1)  # indexing babel
    # set up forcefield
    forcefield = openbabel.OBForceField.FindForceField("MMFF94")
    forcefield.Setup(OBMol_thing, constr)
    # force field optimize structure
    forcefield.ConjugateGradients(2500)
    forcefield.GetCoordinates(OBMol_thing)
    # reset atomic number to metal
    for i, iiat in enumerate(indmtls):
        OBMol_thing.GetAtomById(iiat).SetAtomicNum(mtlsnums[i])
    obConversion.WriteFile(OBMol_thing, 'ff.xyz')

In [46]:
obConversion.WriteFile(OBMol_thing, 'ff.xyz')

True

In [4]:
from openbabel import openbabel

obConversion = openbabel.OBConversion()
obConversion.SetInFormat("xyz")

OBMol = openbabel.OBMol()
obConversion.ReadFile(OBMol, "save_as_opt_axial.xyz")

OBMol_thing = OBMol

metals = list(range(21, 31))+list(range(39, 49))+list(range(72, 81))

constr = openbabel.OBFFConstraints()
# openbabel indexing starts at 1 ### !!!
# convert metals to carbons for FF
indmtls = []
mtlsnums = []
for iiat, atom in enumerate(openbabel.OBMolAtomIter(OBMol_thing)):
    if atom.GetAtomicNum() in metals:
        indmtls.append(iiat)
        mtlsnums.append(atom.GetAtomicNum())
        atom.SetAtomicNum(6)
for cat in range(0, 4):
    constr.AddAtomConstraint(cat+1)  # indexing babel
# set up forcefield
forcefield = openbabel.OBForceField.FindForceField("MMFF94")
forcefield.Setup(OBMol_thing, constr)
# force field optimize structure
forcefield.ConjugateGradients(2500)
forcefield.GetCoordinates(OBMol_thing)
# reset atomic number to metal
for i, iiat in enumerate(indmtls):
    OBMol_thing.GetAtomById(iiat).SetAtomicNum(mtlsnums[i])
obConversion.WriteFile(OBMol_thing, 'ff.xyz')

True